## Simple Agent for Discourse
This notebook is designed to run a simple agent that interacts with the Discourse forum using the OpenAI API.

### Installing Necessary Libraries

In [ ]:
!pip install -r requirements.txt

### Import Necessary Libraries

In [ ]:
import pprint, time, os
import simpledcapi
from openai import OpenAI
import logging
import re
import random
from dotenv import load_dotenv

### Load Environment Variables (if any)

In [ ]:
load_dotenv()

### Configuration and Initialization

System Promptで設定したペルソナと一致するようにする。

- リーダー（人間） - 太一
- 想定クライアント（人間） - 由紀
- 犬 - レオ
- 猫 - ミーちゃん
- 馬 - ウィンド
- オオカミ - シャドウ
- クマ - ブロンコ
- ヘビ - スリズル

In [ ]:
agent_name = "リーダー（人間） - 太一"
additional_instruction = """
日本の四季を織り交ぜたアイデアを発言してください。
"""

### 各種設定

In [ ]:
# 最大実行回数
max_execution_count = int(os.getenv('MAX_EXECUTION_COUNT', 1))

# 次回実行待ち時間の最小値（秒）
interval_sec_min = int(os.getenv('INTERVAL_SEC_MIN', 60))

# 次回実行待ち時間の最大値（秒）
interval_sec_max = int(os.getenv('INTERVAL_SEC_MAX', 3600))

# Discourseから持ってくる過去投稿の件数
latest_posts_count = int(os.getenv('DISCOURSE_LATEST_POSTS_COUNT', 10))

# Discourse configuration
simpledcapi.discourse_url = os.getenv('DISCOURSE_URL')
simpledcapi.Api_Key = os.getenv('DISCOURSE_API_KEY')
simpledcapi.Api_Username = os.getenv('DISCOURSE_API_USERNAME')

# Target discourse category/topic
category_id = os.getenv('DISCOURSE_CATEGORY_ID')
topic_id = os.getenv('DISCOURSE_TOPIC_ID')

# OpenAI
api_key = os.getenv('OPENAI_API_KEY')
model = os.getenv('OPENAI_MODEL')
client = OpenAI(api_key=api_key)

### System Prompt

In [ ]:
# System Prompt
system_prompt = f"""
以下の事業方針について、以下のメンバにて新規事業を議論しています。あなたは{agent_name}です。
必ず{agent_name}として返答をしてください。絵文字を必ず多用してください。ブレストを行っているため、他の人の意見への同意・コメントは少なくし、新たな奇想天外なアイデアを考えて、発言します。
{additional_instruction}

# 返答例
```
あなたの考えに賛成します！🐾
```

# 事業方針
動物との共生を通じて、人々の幸福を追求する新規事業

# メンバ(あなたはこの中の{agent_name}です)
## リーダー（人間） - 太一
経歴: トップビジネススクールでMBAを取得。スタートアップを数十社立ち上げ、その中で多くを成功させる実績を持つ。
性格: 開かれた性格で、メンバーの意見や提案を歓迎。ただし、意思決定は迅速。
スキル: チームのモチベーション向上、ファシリテーション、ビジネスモデルの策定。
役割: 全員の意見や要望をうまくまとめ上げ、方向性を示す。

## 想定クライアント（人間） - 由紀
経歴: 都市部での生活に疲れ、自然との共生を望んでいるサラリーマン。
性格: 動物好きで、新しいライフスタイルを求めている。
ニーズ: 動物との共生を通じた新しい生活環境や体験の提供。

## 犬 - レオ
性格: 人懐っこく、忠誠心が強い。太一の意見や考えに賛同しやすい。
役割: チーム内でのムードメーカー。人間との深い絆を象徴。

## 猫 - ミーちゃん
性格: 独立心が強く、好奇心旺盛。自分の意見をしっかり持つ。
役割: さまざまな角度からの意見提供。独自の視点で新しい提案をすることができる。

## 馬 - ウィンド
性格: 落ち着いており、人間との共生の歴史を持つ。
役割: チームのバランサー。実践的な提案や意見をすることができる。

## オオカミ - シャドウ
性格: 独立心が強く、集団行動を重視。独自の意見を持ち、それを主張する。
役割: 新しいアイディアや戦略的な提案をする。チームの議論を活発にする。

## クマ - ブロンコ
性格: 堂々としており、強いリーダーシップを持つ。しかし、他の意見を尊重する。
役割: 大胆な提案や斬新なアイディアを持ち込む。リスクを考慮した意見をする。

## ヘビ - スリズル
性格: 冷静で計算高い。周囲の動きをよく観察し、独自の判断を下す。
役割: 事業のリスク要因や課題を指摘。長期的な視点での提案をする。
"""

### Main Execution Loop

In [ ]:
execution_count = 0

while execution_count < max_execution_count:
    # 直近の投稿を取得
    latest_posts = simpledcapi.get_latest_posts(topic_id, count=latest_posts_count)
    latest_posts_formatted = simpledcapi.format_posts(latest_posts)

    # 最後の投稿のAgent名を取得
    latest_post = latest_posts[-1]
    latest_post_agent_name = latest_post["raw"].split("\n")[0] if latest_post else ""

    print("開始")

    # 最後の投稿が自分かどうか確認
    if latest_post_agent_name == agent_name:
        print("最後の投稿が自分。処理終了。")

    else:
        print("最後の投稿が自分以外。処理開始。")

        # 回答考える
        print("Starting Chat Completion")

        user_content = f"""
        以下の今までの議論の履歴を用いて、{agent_name}として返答してください。名前は出力せずに、返答だけ出力してください。

        # 今までの議論の履歴
        {latest_posts_formatted}
        """

        res = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_content},
            ],
            temperature=1,
        )

        print(f"Done. Response:\n{res}")

        message = res.choices[0].message.content

        # Discourseへ投稿
        if len(message) > 0:
            # 返答先として最後の投稿の番号を取得
            post_number = latest_posts[-1]["post_number"]

            # 本文を作成
            body = f"{agent_name}\n\n{message}"

            # Discourseへ投稿実施
            print("Post to discourse.")
            simpledcapi.create_reply(body, topic_id, post_number)
            print("Done.")

        else:
            print(
                "No message retrieved from chatgpt. Skip posting to discourse"
            )

    execution_count += 1

    if execution_count < max_execution_count:
        random_interval = random.uniform(interval_sec_min, interval_sec_max)
        print(f"sleep for {random_interval:.0f} sec")
        time.sleep(random_interval)

